In [ ]:
from query.datasets.prelude import *
from tempfile import NamedTemporaryFile as NamedTemp

In [ ]:
def fmt_time(t):
    return '{:02d}:{:02d}:{:02d}.{:03d}'.format(
        int(t / 3600), int(t / 60 % 60), int(t % 60), int(t * 1000 % 1000))


def cut_audio(seg):
    sp.check_call('ffmpeg -y -i {} -ss {} -t {} /tmp/seg.wav'.format(
        '/tmp/out.wav',
        fmt_time(seg.min_frame / video.fps),
        fmt_time((seg.max_frame - seg.min_frame) / video.fps)),
        shell=True)


CLUSTER_SCRIPT_DIR = '/app/deps/speaker-diarization'


def cluster_speakers(video, debug=False):
    if Speaker.objects.filter(video=video, identity__isnull=False).exists():
        return
    
    speakers = list(Speaker.objects.filter(video=video, labeler__name='lium').order_by('min_frame'))
    commercials = list(Commercial.objects.filter(video=video).order_by('min_frame'))
    speakers_nocom = [
        s for s in speakers
        if not any([c.min_frame < s.max_frame and c.max_frame > s.min_frame for c in commercials])]

    with WithMany(pushd(CLUSTER_SCRIPT_DIR), NamedTemp(suffix='.wav'), NamedTemp(suffix='.wav', delete=not debug),
                  named_temp_dir(delete=not debug), NamedTemp(delete=not debug), NamedTemp(delete=not debug)) as \
        (_, audio1f, audio2f, featd, inf, outf):
            
        with Timer('download', run=debug):
            video.extract_audio(output_path=audio1f.name)
        with Timer('convert', run=debug):
            sp.check_call('ffmpeg -y -i {} -ar 16000 -ac 1 {}'.format(audio1f.name, audio2f.name), shell=True)
        with Timer('features', run=debug):
            sp.check_call('./spk-diarization2.py {} -fp {}'.format(audio2f.name, featd), shell=True)

        for i, s in enumerate(speakers_nocom):
            parts = {
                'audio': audio2f.name,
                'lna': 'a_{}'.format(i),
                'start-time': s.min_frame/video.fps,
                'end-time': s.max_frame/video.fps,
                'speaker': 'spk_turn'
            }
            inf.write(' '.join(['{}={}'.format(k, v)
                                for k, v in parts.iteritems()]) + '\n')
        inf.flush()

        with Timer('cluster', run=debug):
            print(sp.check_output(
                './spk-clustering.py {} {} -o {} -m hi -l 1.3'.format(inf.name, featd, outf.name),
                shell=True))

        clusters = readlines(outf.name)

    identities = {}
    for line in clusters:
        parts = [s.split('=') for s in line.split(' ')]
        dct = {p[0]: p[1] for p in parts}
        if not dct['speaker'] in identities:
            identities[dct['speaker']] = Identity(name=dct['speaker'])
    Identity.objects.bulk_create(identities.values())
    
    for line, speaker in zip(clusters, speakers_nocom):
        parts = [s.split('=') for s in line.split(' ')]
        dct = {p[0]: p[1] for p in parts}
        speaker.identity = identities[dct['speaker']]
    Speaker.objects.bulk_update(speakers, update_fields=['identity'])

In [ ]:
# video = Video.objects.get(path='tvnews/videos/FOXNEWSW_20160522_070000_Fox_News_Reporting.mp4')
# with Timer('overall'):
#     cluster_speakers(video, debug=True)

In [ ]:
videos = list(Video.objects.annotate(c=Subquery(Speaker.objects.filter(video=OuterRef('pk')).values('video').annotate(c=Count('video')).values('c'))).filter(c__gt=0))

In [ ]:
with Timer('all'):
    par_for(cluster_speakers, videos[10:20], workers=4)

In [ ]:
# Identity.objects.all().delete()
# Speaker.objects.filter(labeler=labeler).delete()